In [4]:
# %reload_ext autoreload
# %autoreload 2
# from IPython.display import HTML
import pandas as np
from race import Race
from onerace import OneRace

In [3]:
Race().today()

2021年6月23日(水) 伊勢崎 14:21 ('20210623','伊勢崎')
2021年6月24日(木) 伊勢崎 14:21 ('20210624','伊勢崎')
2021年6月25日(金) 伊勢崎 14:21 ('20210625','伊勢崎') *
2021年6月26日(土) 伊勢崎 14:19 ('20210626','伊勢崎')
2021年6月27日(日) 伊勢崎 ??:?? ('20210627','伊勢崎')


In [17]:
race = OneRace('20210625','伊勢崎', 2)
# r.setPrediction()

一般戦 2021年6月25日(金) 伊勢崎 2R 14:51


In [18]:
pd.set_option('display.max_columns', 22)
race.dspEntry()

no,name,age,rank,prnk,vc,point,hand,machine,team,try,avt,avg,max,prd,agm,mxm,pdm
1,北爪 勝義,48歳-23期,A-222,(B-11),V0-0回-V2,58.73,0m,アクティブ７,伊勢崎,3.40,3.40,3.50,3.46,3.50,73.41,55.06,65.15
2,中村 浩章,52歳-22期,A-174,(A-179),V0-0回-V1,63.08,0m,サムライ・１,伊勢崎,3.40,3.38,3.46,3.43,3.48,64.24,14.68,43.13
3,柴田 紘志,39歳-28期,A-181,(A-201),V0-0回-V0,62.49,10m,メゾン,浜松,3.42,3.38,3.48,3.45,3.52,82.59,26.61,73.41
4,新井 裕貴,32歳-30期,A-164,(A-220),V0-1回-V2,64.49,10m,Ａ・Ｋワカチコ,伊勢崎,3.39,3.39,3.48,3.45,3.48,55.06,27.53,37.62
5,木村 義明,53歳-22期,A-89,(A-61),V0-0回-V6,74.35,10m,アイギス,伊勢崎,3.40,3.38,3.48,3.45,3.50,64.24,29.36,57.81
6,竹本 修,47歳-25期,A-117,(A-102),V0-1回-V4,69.08,10m,オサムジャパン,伊勢崎,3.37,3.35,3.46,3.44,3.48,36.71,7.34,35.79
7,野本 佳章,33歳-34期,A-155,(A-230),V1-4回-V1,65.31,20m,キリン,伊勢崎,3.33,3.37,3.48,3.44,3.44,0.00,26.61,0.00
8,伊藤 正真,30歳-33期,A-34,(A-123),V0-3回-V2,81.18,20m,レオーマン,伊勢崎,3.36,3.35,3.45,3.41,3.46,27.53,0.00,19.27


In [19]:
race.dspResult()

no,name,age,rank,prnk,vc,point,hand,machine,team,try,avt,avg,max,prd,run,agm,mxm,pdm,rnm,odr,fav
1,北爪 勝義,48歳-23期,A-222,(B-11),V0-0回-V2,58.73,0m,アクティブ７,伊勢崎,3.40,3.40,3.50,3.46,3.50,3.49,73.41,55.06,65.15,3182.06,1,6
2,中村 浩章,52歳-22期,A-174,(A-179),V0-0回-V1,63.08,0m,サムライ・１,伊勢崎,3.40,3.38,3.46,3.43,3.48,3.49,64.24,14.68,43.13,3182.06,2,2
3,柴田 紘志,39歳-28期,A-181,(A-201),V0-0回-V0,62.49,10m,メゾン,浜松,3.42,3.38,3.48,3.45,3.52,3.50,82.59,26.61,73.41,3199.64,6,5
4,新井 裕貴,32歳-30期,A-164,(A-220),V0-1回-V2,64.49,10m,Ａ・Ｋワカチコ,伊勢崎,3.39,3.39,3.48,3.45,3.48,3.55,55.06,27.53,37.62,3244.46,7,3
5,木村 義明,53歳-22期,A-89,(A-61),V0-0回-V6,74.35,10m,アイギス,伊勢崎,3.40,3.38,3.48,3.45,3.50,3.50,64.24,29.36,57.81,3196.90,5,8
6,竹本 修,47歳-25期,A-117,(A-102),V0-1回-V4,69.08,10m,オサムジャパン,伊勢崎,3.37,3.35,3.46,3.44,3.48,0.00,36.71,7.34,35.79,0.00,-,7
7,野本 佳章,33歳-34期,A-155,(A-230),V1-4回-V1,65.31,20m,キリン,伊勢崎,3.33,3.37,3.48,3.44,3.44,3.47,0.00,26.61,0.00,3187.91,3,1
8,伊藤 正真,30歳-33期,A-34,(A-123),V0-3回-V2,81.18,20m,レオーマン,伊勢崎,3.36,3.35,3.45,3.41,3.46,3.47,27.53,0.00,19.27,3187.91,4,4


In [29]:
_df = race.get_dfs(race.result_soup)[0]
df = _df.sort_values('車')
df.reset_index(inplace=True, drop=True)
sr_lap, sr_odr, sr_fav = df["競走タイム"], df["着"], df["人気"]
sr_odr.name, sr_fav.name = "odr", "fav"
handicaps = [int(h.strip("m")) for h in df["ハンデ"]]
goal_times = [lap * (31 + hdc/100) for hdc, lap in zip(handicaps, sr_lap)]
goals = [t for t in goal_times if t != 0.00] # 失格など
top_time = min(goals)
mps = 0.034
goal_diffs = [(t - top_time) / mps if t != 0.00 else "-" for t in goal_times] # しっか
sr_run = pd.Series(goal_diffs, name="rnm")

entry_df = race.entry()
laps = [lap if lap != 0.00 else "-" for lap in sr_lap]
entry_df.insert(15, "run", laps)
rlt_df = pd.concat([sr_run, sr_odr, sr_fav], axis=1)

result_df = pd.concat([entry_df, rlt_df], axis=1)
result_df

,no,name,age,rank,prnk,vc,point,hand,machine,team,try,avt,avg,max,prd,run,agm,mxm,pdm,rnm,odr,fav
0,1,北爪 勝義,48歳-23期,A-222,(B-11),V0-0回-V2,58.73,0m,アクティブ７,伊勢崎,3.40,3.40,3.50,3.46,3.50,3.49,73.41,55.06,65.15,0.0,1,6
1,2,中村 浩章,52歳-22期,A-174,(A-179),V0-0回-V1,63.08,0m,サムライ・１,伊勢崎,3.40,3.38,3.46,3.43,3.48,3.49,64.24,14.68,43.13,0.0,2,2
2,3,柴田 紘志,39歳-28期,A-181,(A-201),V0-0回-V0,62.49,10m,メゾン,浜松,3.42,3.38,3.48,3.45,3.52,3.5,82.59,26.61,73.41,17.58,6,5
3,4,新井 裕貴,32歳-30期,A-164,(A-220),V0-1回-V2,64.49,10m,Ａ・Ｋワカチコ,伊勢崎,3.39,3.39,3.48,3.45,3.48,3.55,55.06,27.53,37.62,62.4,7,3
4,5,木村 義明,53歳-22期,A-89,(A-61),V0-0回-V6,74.35,10m,アイギス,伊勢崎,3.40,3.38,3.48,3.45,3.50,3.5,64.24,29.36,57.81,14.84,5,8
5,6,竹本 修,47歳-25期,A-117,(A-102),V0-1回-V4,69.08,10m,オサムジャパン,伊勢崎,3.37,3.35,3.46,3.44,3.48,-,36.71,7.34,35.79,-,-,7
6,7,野本 佳章,33歳-34期,A-155,(A-230),V1-4回-V1,65.31,20m,キリン,伊勢崎,3.33,3.37,3.48,3.44,3.44,3.47,0.00,26.61,0.00,5.85,3,1
7,8,伊藤 正真,30歳-33期,A-34,(A-123),V0-3回-V2,81.18,20m,レオーマン,伊勢崎,3.36,3.35,3.45,3.41,3.46,3.47,27.53,0.00,19.27,5.85,4,4


In [33]:
r.setOdds()
r.dspRaceTable()

no,name,age,rank,point,hand,machine,belong,平試走,平競争,実試走,予競争,odds,予,st,comment
1,中村 浩章,52歳/22期,A-174,63.08,0m,サムライ・１,伊勢崎,3.38,3.46,3.39,3.47,18.6,,○,逃げる展開は疑問残り
2,荒川 哲也,48歳/23期,A-148,66.32,10m,シアトリカル,伊勢崎,3.37,3.48,3.35,3.47,20.1,,○,最インから先行狙って
3,竹内 正浩,52歳/21期,A-129,68.24,10m,ドクターＪ,伊勢崎,3.37,3.46,3.35,3.44,16.2,▲,◎,飛び出して中村捲るか
4,中野 光公,48歳/25期,A-147,66.4,10m,ハピネス,伊勢崎,3.35,3.44,3.36,3.46,6.3,×,○,差しあるので注意必要
5,梅内 幹雄,56歳/19期,A-173,63.27,20m,アイバーソン,川口,3.34,3.43,3.33,3.44,7.8,,△,後半に捲りで進出する
6,田中 賢,36歳/29期,A-25,83.6,20m,ネズミマウス,伊勢崎,3.34,3.43,3.33,3.44,4.7,△,○,梅内に先手は取れそう
7,岩崎 亮一,45歳/25期,A-1,88.48,30m,ＦＧビゼン,山陽,3.33,3.42,3.31,3.42,2.8,◎,◎,序盤から積極的に攻め
8,佐藤 貴也,36歳/29期,S-12,110.24,30m,スケートラブ,浜松,3.32,3.42,3.32,3.45,2.4,○,◎,カマシ決めると逆転も


総評:自在に攻める岩崎佐藤貴がマークで


In [4]:
import re
import pandas as pd
import json
import seaborn as sns

In [5]:
r.raceTitle()

'稲妻賞開幕 2021年6月23日(水) 伊勢崎 1R 14:21'

In [50]:
entry_df = r.get_df(r.soup)[0]
lst = []
for n in range(len(entry_df)):
    _racer = entry_df.values[n][:6]
    racer = [re.sub("  |\u3000", " ", r) for r in _racer if type(r) == str]
    # print(racer)
    # -> ['城戸 徹 41歳/27期 V0/0回/V1 ヤブキ３７３/1', '飯 塚', 
    # -> '0m 3.89 0.073', 'B-43 (B-46) 53.433', '3.39 3.463 3.423']
    fstname, lstname, _age, _v, _machine = racer[0].split() # 名前、年齢（期）、勝利数、マシン、所属
    name = fstname + " " + lstname
    age = re.sub("/", "-", _age)
    v = re.sub("/", "-", _v)
    machine = re.sub("/1", "", _machine)
    team = racer[1].replace(" ", "")
    # print(name, age, v, machine, team)
    
    handicap, _triTime, _triDev = racer[2].split() # ハンデ、試走タイム、試走偏差、予想タイム
    int_hand = int(handicap.strip("m")) # print(handicap, int_hand)
    if r.is_num(_triTime):
        triTime = float(_triTime)
        prdTime = triTime + float(_triDev)
    else:
        triTime = 0.0
        prdTime = 0.0
    triDev = float(_triDev)
    # print(triTime, triDev, prdTime)
    
    rank, preRank, _point = racer[3].split() # ランク、(前ランク)、審査ポイント
    point = float(_point)
    # print(rank, preRank, point)
    
    _avgTriTime, _avgTime, _maxTime = racer[4].split() # 良10走
    avgTriTime = float(_avgTriTime) # 平均試走T
    avgTime = float(_avgTime) # 平均競走T
    maxTime = float(_maxTime)# 最高競走T
    # print(avgTriTime, avgTime, maxTime)
    
    if n == 0 and not prdTime == 0.0: # 最内コース補正
        avgTime, maxTime, prdTime = avgTime + 0.01, maxTime + 0.01, prdTime + 0.01
    avgRun = avgTime * (31 + int_hand/100)
    maxRun = maxTime * (31 + int_hand/100)
    prdRun = prdTime * (31 + int_hand/100)
    # print(avgRun, maxRun, prdRun)

    # print(avgTrial, avgRun, trial_time, trialDev, pred_time)
    idx = ["no", "name", "age", "rank", "prnk", "vc", "point", "hand", "machine", "team"]
    dat = [str(n+1), name, age, rank, preRank, v, point, handicap, machine, team]
    idx += ["try", "avg", "max", "prd", "avg run", "max run", "prd run"]
    dat += [triTime, avgTime, maxTime, prdTime, avgRun, maxRun, prdRun]


    sr = pd.Series(dat, index=idx)
    lst.append(sr)
#     odds = r.odds_d[n]
#     pred = r.pred_d[n]
df = pd.DataFrame(lst)

avg_sr, max_sr, prd_sr = df["avg run"], df["max run"], df["prd run"]
df_drp = df.drop(["avg run", "max run", "prd run"], axis=1)
sr_lst = []
for sr in [avg_sr, max_sr, prd_sr]:
    min_v = min(sr)
    lst = [(v - min_v) / 0.034 for v in sr]
    new_name = sr.name.split()[0] + "-m"
    new_sr = pd.Series(lst, index=list(range(8)), name=new_name)
    sr_lst.append(new_sr)

race_df = pd.concat([df_drp, *sr_lst], axis=1)

cm = sns.light_palette("green", as_cmap=True)
display(race_df.style.background_gradient(cmap=cm).hide_index())

p = "../../ruby/gosu/gosu_race/test.json"
race_df.to_json(p, force_ascii=False)
# print("saved:", path)

no,name,age,rank,prnk,vc,point,hand,machine,team,try,avg,max,prd,avg-m,max-m,prd-m
1,内越 忠徳,63歳-14期,A-218,(B-24),V0-0回-V10,59.14,0m,Ｍシフリン,伊勢崎,3.42,3.50,3.48,3.51,43.76,54.71,62.00
2,佐藤 大地,24歳-34期,A-200,(B-58),V1-1回-V1,60.95,0m,エルデ,浜松,3.35,3.46,3.42,3.45,0.00,0.00,0.00
3,栗原 勝測,57歳-20期,A-152,(A-152),V0-0回-V8,65.65,10m,ビジュアルＫ,伊勢崎,3.41,3.48,3.46,3.49,35.78,42.08,44.90
4,押田 和也,44歳-27期,A-168,(A-154),V0-0回-V1,63.80,10m,ディープＲＥＤ,伊勢崎,3.36,3.45,3.44,3.45,7.42,27.45,16.54
5,落合 淳,41歳-29期,A-176,(A-133),V0-0回-V3,62.73,10m,ブルズ,伊勢崎,3.37,3.48,3.45,3.49,26.63,33.85,44.90
6,田方 秀和,52歳-22期,A-158,(A-50),V0-1回-V10,64.82,10m,ラファエラ,山陽,3.39,3.47,3.43,3.47,24.80,15.56,33.92
7,五所 淳,48歳-23期,A-111,(A-7),V0-1回-V9,70.25,20m,パンドラー,山陽,3.39,3.47,3.45,3.48,29.51,42.16,47.81
8,大月 渉,33歳-31期,A-56,(A-59),V0-1回-V2,78.11,20m,キラークイーン,伊勢崎,3.38,3.45,3.43,3.48,11.16,29.31,47.81


In [51]:
_, result_df = r.reqResult()
zero_lst = [0.00 for _ in range(len(entry_df))]
r_sr = pd.Series(zero_lst, name="run")
if len(result_df):
    r_sr = result_df["競走タイム"]
    r_sr.name = "run"

min_v = min(r_sr)
lst = [(v - min_v) / 0.034 for v in r_sr]
rm_sr = pd.Series(lst, index=list(range(8)), name="run-m")
print(r_sr)
print(rm_sr)

0    3.46
1    3.47
2    3.50
3    3.50
4    3.50
5    3.49
6    3.50
7    3.53
Name: run, dtype: float64
0    0.00
1    0.35
2    1.09
3    1.21
4    1.29
5    0.97
6    1.15
7    2.00
Name: run-m, dtype: float64


In [8]:
p = "../../ruby/gosu/gosu_race/test2.json"
d = {}
d["title"] = r.raceTitle()
with open(p, 'w') as f:
    json.dump(d, f, ensure_ascii=True)

In [30]:
_, r_df = r.reqResult()
if not len(r_df):
    print(1)
# r_df["競走タイム"]

1


In [41]:
r.dspResult()

,入賞,配当,人気
単勝,7,280円,2番人気
2連複,2 - 7,"3,540円",16番人気
2連単,7 - 2,"5,480円",21番人気
3連複,2 - 3 - 7,"8,530円",36番人気
3連単,7 - 2 - 3,"32,640円",112番人気


着,事故,車,選手名,年齢/期,LG,ハンデ,試走タイム,競走タイム,ST,異常,人気
1,nan,7,岩崎 亮一,45歳/25期,山陽,30m,3.31,3.42,0.14,nan,2
2,nan,2,荒川 哲也,48歳/23期,伊勢崎,10m,3.35,3.44,0.07,nan,8
3,nan,3,竹内 正浩,52歳/21期,伊勢崎,10m,3.35,3.45,0.16,nan,6
4,nan,8,佐藤 貴也,36歳/29期,浜松,30m,3.32,3.43,0.13,nan,1
5,nan,5,梅内 幹雄,56歳/19期,川口,20m,3.33,3.45,0.10,nan,5
6,nan,1,中村 浩章,52歳/22期,伊勢崎,0m,3.39,3.47,0.23,nan,7
7,nan,6,田中 賢,36歳/29期,伊勢崎,20m,3.33,3.45,0.13,nan,3
8,nan,4,中野 光公,48歳/25期,伊勢崎,10m,3.36,3.48,0.15,nan,4


In [35]:
r.soup.title

<title>【レース別出走表】ランチアタック準決勝戦｜2021年6月21日(月) 浜松 5R｜オートレースならオッズパークオートレース</title>

In [7]:
# race.setAiPrediction()
# race.dsp5raceTable()

In [69]:
HTML(r.to_html())

準々決勝戦｜2021年6月20日(日)11R


In [16]:
race.dspQuinella()

車券,配当
1=2,63.7
1=3,31.8
1=4,31.8
1=5,95.5
1=6,63.7
1=7,63.7
1=8,17.3
2=3,11.9
2=4,38.2
2=5,47.7


In [8]:
race.dspResult()

stop run!


In [2]:
Results().results("20210519", "伊勢崎")

holding,no,name,age,belong,handi,trial,race,単人気,3連単,3人気
2021年5月19日(水) 伊勢崎 1R,3,室田 泰利,48歳/24期,飯塚,10m,3.85,3.91,1番人気,"1,810円",1番人気
2021年5月19日(水) 伊勢崎 2R,6,竹中 修二,45歳/26期,飯塚,10m,3.90,3.92,2番人気,"3,270円",13番人気
2021年5月19日(水) 伊勢崎 3R,3,清岡 優一,42歳/29期,伊勢崎,10m,3.83,3.92,1番人気,410円,1番人気
2021年5月19日(水) 伊勢崎 4R,4,高橋 絵莉子,25歳/33期,伊勢崎,10m,3.83,3.94,1番人気,"8,960円",34番人気
2021年5月19日(水) 伊勢崎 5R,8,加賀谷 建明,43歳/27期,川口,40m,3.81,3.86,1番人気,720円,2番人気
2021年5月19日(水) 伊勢崎 6R,8,田中 賢,36歳/29期,伊勢崎,20m,3.81,3.89,1番人気,"1,160円",5番人気
2021年5月19日(水) 伊勢崎 7R,3,北渡瀬 充,46歳/25期,伊勢崎,10m,3.85,3.88,1番人気,800円,1番人気
2021年5月19日(水) 伊勢崎 8R,1,小林 晃,42歳/26期,伊勢崎,0m,3.84,3.88,3番人気,"3,170円",6番人気
2021年5月19日(水) 伊勢崎 9R,2,池田 康範,54歳/21期,山陽,10m,3.77,3.80,2番人気,980円,4番人気
2021年5月19日(水) 伊勢崎 10R,2,大月 渉,33歳/31期,伊勢崎,10m,3.71,3.72,2番人気,"7,210円",25番人気


In [12]:
soup = Results().request('20210521','川口', 12)
Results().quine(soup)

0    2021年5月21日(金) 川口 12R
1                   -3000
dtype: object

In [35]:
df = Results().quineTest('20210520','川口')

holding,blance
2021年5月20日(木) 川口 1R,-3000
2021年5月20日(木) 川口 2R,-3000
2021年5月20日(木) 川口 3R,-3000
2021年5月20日(木) 川口 4R,-3000
2021年5月20日(木) 川口 5R,12900
2021年5月20日(木) 川口 6R,-3000
2021年5月20日(木) 川口 7R,-500
2021年5月20日(木) 川口 8R,2700
2021年5月20日(木) 川口 9R,5900
2021年5月20日(木) 川口 10R,-2000


blance    4300
dtype: int64


In [68]:
dfs = race.get_df(soup)
# dfs[0]["車"][:3]
dfs[0]

,着,事故,車,選手名,年齢/期,LG,ハンデ,試走タイム,競走タイム,ST,異常,人気
0,1,NaN,7,丹村 飛竜,37歳/29期,山陽,20m,3.44,3.42,0.20,NaN,3
1,2,NaN,4,鈴木 清,48歳/23期,川口,20m,3.44,3.42,0.18,NaN,2
2,3,NaN,2,平川 博康,42歳/28期,川口,0m,3.51,3.45,0.01,NaN,8
3,4,NaN,6,佐藤 裕二,47歳/24期,川口,20m,3.48,3.43,0.06,NaN,6
4,5,NaN,1,高塚 義明,42歳/29期,川口,0m,3.52,3.46,0.10,NaN,5
5,6,NaN,3,間中 大輔,43歳/28期,川口,10m,3.44,3.45,0.15,NaN,4
6,7,NaN,8,永井 大介,44歳/25期,川口,20m,3.41,3.44,0.12,NaN,1
7,8,NaN,5,中山 光,33歳/32期,川口,20m,3.53,3.48,0.24,NaN,7


In [75]:
prize = 0
if set(dfs[0]["人気"][:2]) <= {1, 2, 3}:
    prize_s = dfs[2].set_index(0).at["2連複", 2 ]
    prize = int(prize_s.strip("円"))

In [80]:
lst = [soup.find("title").text.split("｜")[1]]
# lst += [dfs[0].iloc[0, i] for i in [2, 3, 4, 5, 6, 7, 8]]
lst

['2021年5月21日(金) 川口 11R']

In [15]:
dic = race.reqOdds()

In [37]:
sorted([float(v) for v in dic.values()])

[1.3, 3.6, 7.3, 10.5, 12.1, 28.9, 31.3, 45.1]

In [40]:
def get_key_from_value(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None
lst = sorted([float(dic[d]) for d in dic])[:3]
[get_key_from_value(dic, str(e)) for e in lst ]

[8, 4, 7]

In [17]:
def result(soup):
    if soup.find("table") == None:
        print("no table in the soup object.")
        return
    dfs = get_df(soup)
    d3 = len(dfs[3])
    d = dfs[3].loc[[d3-1, ], :]
    lst = [soup.find("title").text.split("｜")[1]]
    lst += [dfs[0].iloc[0, i] for i in [2, 3, 4, 5, 6, 7, 8]]
    lst += [str(dfs[0].iloc[0, 11]) + "番人気"]
    lst += [d.iloc[0, 2], d.iloc[0, 3]]
    sr = pd.Series(lst)
    return sr
result(soup)

0     2021年5月19日(水) 伊勢崎 12R
1                         3
2                      田中 哲
3                   40歳/28期
4                       伊勢崎
5                       20m
6                      3.69
7                      3.68
8                      4番人気
9                   57,580円
10                   157番人気
dtype: object

In [34]:
dic = {1: ("◯", "c", "c"), 2: ("▲", "d", "d"), 3: ("△", "e", "e")}
swp_d = {v[0]: (k, v[1], v[2]) for k, v in dic.items() if not v[0] == ""}
print(*swp_d["◯"])

1 c c


In [10]:
strr = "12 43 78"
a, b, c = strr.split()
a

'12'